# LIBRARIES IMPORT

In [2]:
# imports

import fig_lib 
import CNN_lib
import CNN_utilities
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import pickle


In [3]:
init_time = time.time()

In [4]:
# p
pkl_d_p = 'C:\PROJECTS\emboendo\CNN/input_d.pkl'
pkl_train_p='C:\PROJECTS\emboendo\CNN/train_d.pkl'        

In [5]:
# ------- PROCESS DICOM DATA --------------

data_load_dicom,generate_new_peakle=True,True

max_frames_allowed=40

# ------- GENERATE SYNTHETIC DATA --------------

generate_new_data=False 


# --------- LOAD TRAN DATA ------------

load_train_data=False

# --------- DATA TYPE ------------
video_d=True                                    # Video = True , Image = False 

# DATA LOAD

In [6]:

if data_load_dicom:

    patients_recs_d_df=CNN_utilities.main_d_df()
    patients_labels_d_df=CNN_utilities.labels_df()
    patients_d_df=pd.merge( patients_labels_d_df,patients_recs_d_df, on='PatientID')
    

    tot_recs=sum(len(records) for records in patients_d_df["Records"])
    filtered_df_0,filtered_df_1 = patients_d_df[patients_d_df['label'] == 0],patients_d_df[patients_d_df['label'] == 1]
    label_0,label_1 = sum(len(records) for records in filtered_df_0['Records']),sum(len(records) for records in filtered_df_1['Records'])

    print(f'- PATIENTS: {patients_d_df.shape[0]} ,PATIENT LABEL 0: {sum(patients_d_df["label"]==0)}, PATIENT LABEL 1: {sum(patients_d_df["label"]==1)}')
    print( f'- RECORDS: {tot_recs} , RECORDS LABEL 0: {label_0}, RECORDS LABEL 1:{label_1}')
    print(f'- RATIO LABEL 0: {np.round(label_0/sum(patients_d_df["label"]==0),decimals=2)}, RATIO LABEL 1: {np.round(label_1/sum(patients_d_df["label"]==1),decimals=2)}')

    display(patients_d_df)


- PATIENTS: 65 ,PATIENT LABEL 0: 47, PATIENT LABEL 1: 18
- RECORDS: 574 , RECORDS LABEL 0: 411, RECORDS LABEL 1:163
- RATIO LABEL 0: 8.74, RATIO LABEL 1: 9.06


,PatientID,Sex,Age,S.E.,M.A.,A.C.V.,label,Records
0,62,0,67,0,0,0,0,"[I0, I1, I2, I9, I11]"
1,76,1,72,0,0,0,0,"[I24, I25, I26, I27, I28, I33, I38, I39, I40, ..."
2,93,0,78,0,0,0,0,"[I50, I51, I53, I54, I56, I57, I58, I61, I63, ..."
3,103,0,82,1,0,0,1,"[I70, I71, I72, I73, I77]"
4,105,1,57,1,0,0,1,"[I86, I87, I89, I90, I91, I92, I93, I94, I95, ..."
...,...,...,...,...,...,...,...,...
60,1311,1,73,0,0,0,0,"[I1183, I1189, I1190, I1192, I1197, I1200, I1202]"
61,1327,1,55,0,0,0,0,"[I1203, I1205, I1206, I1207, I1209, I1215, I12..."
62,1340,1,73,0,0,0,0,"[I1221, I1224, I1226, I1227, I1228, I1232, I12..."
63,1358,1,62,1,0,0,1,"[I1242, I1243, I1244, I1247, I1248, I1252, I12..."


In [7]:

if data_load_dicom:
    
    if generate_new_peakle:
        
        input_d={}    

        for label,ii in zip(patients_d_df['label'],patients_d_df['Records']):
            
            for rec in ii:

                input_d[rec]={'label':label}
                ds_file=CNN_utilities.load_rec(rec)
                vd_ls=[]

                for i in range(ds_file.pixel_array.shape[0]):
                    
                    if i== max_frames_allowed: break
                    else:
                        image_8bit,  width, height = CNN_utilities.im_data(ds_file.pixel_array[i])
                        vd_ls.append(image_8bit)
                    

                input_d[rec]['image']=np.array(vd_ls)
                input_d[rec]['dimHW']= [height,width]   

        with open(pkl_d_p, 'wb') as pikle_file:
            pickle.dump(input_d, pikle_file)
        

    else: 

        with open(pkl_d_p, 'rb') as pikle_file:
            input_d = pickle.load(pikle_file)
       


In [8]:
if data_load_dicom:    
    
    print('FIRST LAYER KEYS:', list(input_d.keys()))
    print('SECOND LAYER KEYS:',list(input_d[list(input_d.keys())[0]].keys()))
    print(f' KEY TYPE "{list(input_d[list(input_d.keys())[0]].keys())[0]}":',type(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[0]].keys())[0]]),'Example:',input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[0]].keys())[0]])
    print(f' KEY TYPE "{list(input_d[list(input_d.keys())[0]].keys())[1]}":',type(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[1]].keys())[1]]),'Example shape:',np.shape(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[1]].keys())[1]]))
    


FIRST LAYER KEYS: ['I0', 'I1', 'I2', 'I9', 'I11', 'I24', 'I25', 'I26', 'I27', 'I28', 'I33', 'I38', 'I39', 'I40', 'I42', 'I44', 'I45', 'I47', 'I49', 'I50', 'I51', 'I53', 'I54', 'I56', 'I57', 'I58', 'I61', 'I63', 'I64', 'I65', 'I66', 'I67', 'I70', 'I71', 'I72', 'I73', 'I77', 'I86', 'I87', 'I89', 'I90', 'I91', 'I92', 'I93', 'I94', 'I95', 'I101', 'I102', 'I103', 'I104', 'I107', 'I111', 'I112', 'I115', 'I116', 'I117', 'I119', 'I120', 'I121', 'I123', 'I124', 'I125', 'I126', 'I127', 'I131', 'I146', 'I147', 'I154', 'I157', 'I159', 'I161', 'I163', 'I164', 'I165', 'I167', 'I168', 'I169', 'I172', 'I174', 'I176', 'I177', 'I178', 'I179', 'I180', 'I181', 'I182', 'I187', 'I189', 'I191', 'I200', 'I201', 'I202', 'I203', 'I204', 'I208', 'I209', 'I212', 'I213', 'I215', 'I216', 'I218', 'I230', 'I238', 'I240', 'I243', 'I252', 'I254', 'I255', 'I257', 'I259', 'I261', 'I262', 'I263', 'I264', 'I265', 'I266', 'I267', 'I268', 'I269', 'I270', 'I278', 'I279', 'I281', 'I283', 'I284', 'I285', 'I286', 'I292', 'I294',

In [9]:
if data_load_dicom:
    print('RECORD - LABEL - SHAPE')

    for R in input_d.keys():
        print(R,input_d[R]['label'],np.shape(input_d[R]['image']),type(input_d[R]['image']))
    

RECORD - LABEL - SHAPE
I0 0 (39, 600, 800) <class 'numpy.ndarray'>
I1 0 (40, 600, 800) <class 'numpy.ndarray'>
I2 0 (40, 600, 800) <class 'numpy.ndarray'>
I9 0 (40, 600, 800) <class 'numpy.ndarray'>
I11 0 (40, 600, 800) <class 'numpy.ndarray'>
I24 0 (23, 434, 636) <class 'numpy.ndarray'>
I25 0 (23, 434, 636) <class 'numpy.ndarray'>
I26 0 (21, 434, 636) <class 'numpy.ndarray'>
I27 0 (23, 434, 636) <class 'numpy.ndarray'>
I28 0 (23, 434, 636) <class 'numpy.ndarray'>
I33 0 (24, 434, 636) <class 'numpy.ndarray'>
I38 0 (24, 434, 636) <class 'numpy.ndarray'>
I39 0 (24, 434, 636) <class 'numpy.ndarray'>
I40 0 (20, 434, 636) <class 'numpy.ndarray'>
I42 0 (22, 434, 636) <class 'numpy.ndarray'>
I44 0 (23, 434, 636) <class 'numpy.ndarray'>
I45 0 (24, 434, 636) <class 'numpy.ndarray'>
I47 0 (23, 434, 636) <class 'numpy.ndarray'>
I49 0 (23, 434, 636) <class 'numpy.ndarray'>
I50 0 (39, 422, 636) <class 'numpy.ndarray'>
I51 0 (39, 422, 636) <class 'numpy.ndarray'>
I53 0 (39, 422, 636) <class 'numpy.n

# DATA GENERATOR

In [10]:
# -------- DATA PARAMS --------               

if generate_new_data:    

    total_n_recs=400
    #n_recs_train,n_recs_eval=300,80
    HEIGHT, WIDTH = 100, 100
    circles_filled=False
    color=255

    # -------- Video ---------
 

    n_frames=[10,12]    # [min,max]

    t_btw_frm=0.2

In [11]:
#input_d, eval_d

if generate_new_data:

    if video_d:
        
        input_d=fig_lib.vid_data_gen(HEIGHT, WIDTH,total_n_recs,n_frames,'I',cir_f=circles_filled,color=color)
        
    else:

        input_d=fig_lib.im_data_gen(HEIGHT, WIDTH,total_n_recs,'I',cir_f=circles_filled,color=color)

    print('FIRST LAYER KEYS:', list(input_d.keys()))
    print('SECOND LAYER KEYS:',list(input_d[list(input_d.keys())[0]].keys()))
    print(f' KEY TYPE "{list(input_d[list(input_d.keys())[0]].keys())[0]}":',type(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[0]].keys())[0]]),'Example:',input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[0]].keys())[0]])
    print(f' KEY TYPE "{list(input_d[list(input_d.keys())[0]].keys())[1]}":',type(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[1]].keys())[1]]),'Example shape:',np.shape(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[1]].keys())[1]]))


In [12]:
print('RECORD - LABEL - SHAPE')

if generate_new_data:
    
    for R in input_d.keys():
        print(R,input_d[R]['label'],np.shape(input_d[R]['image']),type(input_d[R]['image']))

RECORD - LABEL - SHAPE


# CHECK DATA

In [13]:
d_plt_check,rec_check=False,'I0'
t_btw_frm=0.05
print(input_d[rec_check]['label'])


0


In [14]:
# check video

if d_plt_check:

    if video_d:fig_lib.vid_show(input_d[rec_check]['image'],t_btw_frm=t_btw_frm)
    else:fig_lib.im_show(input_d[rec_check]['image'])



# CNN

### GENERATE TRAIN DATA

In [15]:
# X_train,Y_train

if load_train_data: 
      
    with open(pkl_train_p, 'rb') as pikle_file:

        save_trains= pickle.load(pikle_file)

    X_train,Y_train =  save_trains[0],save_trains[1]

else:

    if video_d:X_train,Y_train,max_frm_n,HEIGHT, WIDTH =CNN_lib.vid_d_bin_gen(input_d, zero=0, one= 1)
    else:X_train,Y_train =CNN_lib.im_d_bin_gen(input_d,zero='circle', one= 'line')

    save_trains=[X_train,Y_train]

    with open(pkl_train_p, 'wb') as pikle_file:
            
            pickle.dump(save_trains, pikle_file)



In [16]:
end_time = time.time()

process_time = end_time - init_time
print(f"Tiempo transcurrido: {process_time} segundos")

Tiempo transcurrido: 186.90253138542175 segundos
